* https://huggingface.co/learn/nlp-course/en/chapter6/8?fw=pt
* https://colab.research.google.com/github/huggingface/notebooks/blob/master/course/en/chapter6/section8.ipynb

In [1]:
!pip install datasets evaluate transformers[sentencepiece]

# Acquire a corpus

In [2]:
from datasets import load_dataset

dataset = load_dataset("wikitext", name="wikitext-2-raw-v1", split="train")


def get_training_corpus():
    for i in range(0, len(dataset), 1000):
        yield dataset[i : i + 1000]["text"]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/733k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/6.36M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/657k [00:00<?, ?B/s]

Generating test split:   0%|          | 0/4358 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/36718 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3760 [00:00<?, ? examples/s]

In [3]:
with open("wikitext-2.txt", "w", encoding="utf-8") as f:
    for i in range(len(dataset)):
        f.write(dataset[i]["text"] + "\n")

# Build a WordPiece tokenizer

In [5]:
from tokenizers import (
    Tokenizer,
    decoders,
    models,
    normalizers,
    pre_tokenizers,
    processors,
    trainers,
)

tokenizer = Tokenizer(models.WordPiece(unk_token="[UNK]"))

## Normalizer

In [10]:
tokenizer.normalizer = normalizers.BertNormalizer(lowercase=True)

In [11]:
print(tokenizer.normalizer.normalize_str("\u0085"))

In [12]:
tokenizer.normalizer = normalizers.Sequence(
    [normalizers.NFD(), normalizers.Lowercase(), normalizers.StripAccents()]
)

In [9]:
print(tokenizer.normalizer.normalize_str("Héllò hôw are ü?"))

hello how are u?


In [13]:
print(tokenizer.normalizer.normalize_str("\u0085"))

## Pre-Tokenizer

In [14]:
pre_tokenizer = pre_tokenizers.Sequence(
    [pre_tokenizers.WhitespaceSplit(), pre_tokenizers.Punctuation()]
)
pre_tokenizer.pre_tokenize_str("Let's test my pre-tokenizer.")

[('Let', (0, 3)),
 ("'", (3, 4)),
 ('s', (4, 5)),
 ('test', (6, 10)),
 ('my', (11, 13)),
 ('pre', (14, 17)),
 ('-', (17, 18)),
 ('tokenizer', (18, 27)),
 ('.', (27, 28))]

## Train the tokenizer

In [15]:
special_tokens = ["[UNK]", "[PAD]", "[CLS]", "[SEP]", "[MASK]"]
trainer = trainers.WordPieceTrainer(vocab_size=25000, special_tokens=special_tokens)

In [16]:
tokenizer.train_from_iterator(get_training_corpus(), trainer=trainer)

In [17]:
# Train from the text file.
# tokenizer.model = models.WordPiece(unk_token="[UNK]")
# tokenizer.train(["wikitext-2.txt"], trainer=trainer)

In [18]:
encoding = tokenizer.encode("Let's test this tokenizer.")
print(encoding.tokens)

['l', '##et', "##'s t", '##est ', '##this ', '##to', '##ken', '##iz', '##er', '##.']


## Post-Processing

In [19]:
cls_token_id = tokenizer.token_to_id("[CLS]")
sep_token_id = tokenizer.token_to_id("[SEP]")
print(cls_token_id, sep_token_id)

2 3


In [20]:
tokenizer.post_processor = processors.TemplateProcessing(
    single=f"[CLS]:0 $A:0 [SEP]:0",
    pair=f"[CLS]:0 $A:0 [SEP]:0 $B:1 [SEP]:1",
    special_tokens=[("[CLS]", cls_token_id), ("[SEP]", sep_token_id)],
)

In [21]:
encoding = tokenizer.encode("Let's test this tokenizer.")
print(encoding.tokens)

['[CLS]', 'l', '##et', "##'s t", '##est ', '##this ', '##to', '##ken', '##iz', '##er', '##.', '[SEP]']


In [22]:
encoding = tokenizer.encode("Let's test this tokenizer...", "on a pair of sentences.")
print(encoding.tokens)
print(encoding.type_ids)

['[CLS]', 'l', '##et', "##'s t", '##est ', '##this ', '##to', '##ken', '##iz', '##er', '##...', '[SEP]', 'o', '##n ', '##a pair of ', '##sentenc', '##es', '##.', '[SEP]']
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1]


In [28]:
encoding = tokenizer.encode(
    " ".join(["Let's test this tokenizer...", "on three sentences:", "hello world!"])
)
print(encoding.tokens)
print(encoding.type_ids)

['[CLS]', 'l', '##et', "##'s t", '##est ', '##this ', '##to', '##ken', '##iz', '##er', '##... ', '##on three ', '##sentenc', '##es', '##: ', '##hell', '##o ', '##world', '##!', '[SEP]']
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


## Decoder

In [26]:
tokenizer.decoder = decoders.WordPiece(prefix="##")

In [29]:
tokenizer.decode(encoding.ids)

"let's test this tokenizer... on three sentences: hello world!"

## Push to Hub

In [30]:
tokenizer.save("tokenizer.json")

In [31]:
new_tokenizer = Tokenizer.from_file("tokenizer.json")

In [32]:
from transformers import PreTrainedTokenizerFast

wrapped_tokenizer = PreTrainedTokenizerFast(
    tokenizer_object=new_tokenizer,
    # tokenizer_file="tokenizer.json", # You can load from the tokenizer file, alternatively
    unk_token="[UNK]",
    pad_token="[PAD]",
    cls_token="[CLS]",
    sep_token="[SEP]",
    mask_token="[MASK]",
)

In [33]:
from transformers import BertTokenizerFast

wrapped_tokenizer = BertTokenizerFast(tokenizer_object=tokenizer)

In [34]:
from huggingface_hub import notebook_login

notebook_login()

In [35]:
wrapped_tokenizer.push_to_hub("dummy")

README.md:   0%|          | 0.00/31.0 [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/renwei2024/dummy/commit/59174bb2cb03a3381e4b8c4b4d6d9d9898ce507e', commit_message='Upload tokenizer', commit_description='', oid='59174bb2cb03a3381e4b8c4b4d6d9d9898ce507e', pr_url=None, repo_url=RepoUrl('https://huggingface.co/renwei2024/dummy', endpoint='https://huggingface.co', repo_type='model', repo_id='renwei2024/dummy'), pr_revision=None, pr_num=None)

# Build a BPE tokenizer

In [36]:
tokenizer = Tokenizer(models.BPE())

In [37]:
tokenizer.pre_tokenizer = pre_tokenizers.ByteLevel(add_prefix_space=False)

In [38]:
tokenizer.pre_tokenizer.pre_tokenize_str("Let's test pre-tokenization!")

[('Let', (0, 3)),
 ("'s", (3, 5)),
 ('Ġtest', (5, 10)),
 ('Ġpre', (10, 14)),
 ('-', (14, 15)),
 ('tokenization', (15, 27)),
 ('!', (27, 28))]

In [39]:
trainer = trainers.BpeTrainer(vocab_size=25000, special_tokens=["<|endoftext|>"])
tokenizer.train_from_iterator(get_training_corpus(), trainer=trainer)

In [40]:
# Train from the text file.
# tokenizer.model = models.BPE()
# tokenizer.train(["wikitext-2.txt"], trainer=trainer)

In [41]:
encoding = tokenizer.encode("Let's test this tokenizer.")
print(encoding.tokens)

['L', 'et', "'", 's', 'Ġtest', 'Ġthis', 'Ġto', 'ken', 'izer', '.']


In [42]:
tokenizer.post_processor = processors.ByteLevel(trim_offsets=False)

In [43]:
sentence = "Let's test this tokenizer."
encoding = tokenizer.encode(sentence)
start, end = encoding.offsets[4]
sentence[start:end]

' test'

In [44]:
tokenizer.post_processor = processors.ByteLevel(trim_offsets=True)

In [45]:
sentence = "Let's test this tokenizer."
encoding = tokenizer.encode(sentence)
start, end = encoding.offsets[4]
sentence[start:end]

'test'

In [46]:
tokenizer.decoder = decoders.ByteLevel()

In [47]:
tokenizer.decode(encoding.ids)

"Let's test this tokenizer."

In [48]:
from transformers import PreTrainedTokenizerFast

wrapped_tokenizer = PreTrainedTokenizerFast(
    tokenizer_object=tokenizer,
    bos_token="<|endoftext|>",
    eos_token="<|endoftext|>",
)

In [49]:
from transformers import GPT2TokenizerFast

wrapped_tokenizer = GPT2TokenizerFast(tokenizer_object=tokenizer)

# Build a Unigram tokenizer

In [50]:
tokenizer = Tokenizer(models.Unigram())

In [51]:
from tokenizers import Regex

tokenizer.normalizer = normalizers.Sequence(
    [
        normalizers.Replace("``", '"'),
        normalizers.Replace("''", '"'),
        normalizers.NFKD(),
        normalizers.StripAccents(),
        normalizers.Replace(Regex(" {2,}"), " "),
    ]
)

In [53]:
# https://huggingface.co/docs/tokenizers/api/pre-tokenizers#tokenizers.pre_tokenizers.Metaspace
tokenizer.pre_tokenizer = pre_tokenizers.Metaspace()

In [55]:
tokenizer.pre_tokenizer.pre_tokenize_str("Let's test the   pre-tokenizer!")

[("▁Let's", (0, 5)),
 ('▁test', (5, 10)),
 ('▁the', (10, 14)),
 ('▁', (14, 15)),
 ('▁', (15, 16)),
 ('▁pre-tokenizer!', (16, 31))]

In [56]:
special_tokens = ["<cls>", "<sep>", "<unk>", "<pad>", "<mask>", "<s>", "</s>"]
trainer = trainers.UnigramTrainer(
    vocab_size=25000, special_tokens=special_tokens, unk_token="<unk>"
)
tokenizer.train_from_iterator(get_training_corpus(), trainer=trainer)

In [57]:
# Train from the text file.
# tokenizer.model = models.Unigram()
# tokenizer.train(["wikitext-2.txt"], trainer=trainer)

In [58]:
encoding = tokenizer.encode("Let's test this tokenizer.")
print(encoding.tokens)

['▁Let', "'", 's', '▁test', '▁this', '▁to', 'ken', 'izer', '.']


In [59]:
cls_token_id = tokenizer.token_to_id("<cls>")
sep_token_id = tokenizer.token_to_id("<sep>")
print(cls_token_id, sep_token_id)

0 1


In [60]:
tokenizer.post_processor = processors.TemplateProcessing(
    single="$A:0 <sep>:0 <cls>:2",
    pair="$A:0 <sep>:0 $B:1 <sep>:1 <cls>:2",
    special_tokens=[("<sep>", sep_token_id), ("<cls>", cls_token_id)],
)

In [61]:
encoding = tokenizer.encode("Let's test this tokenizer...", "on a pair of sentences!")
print(encoding.tokens)
print(encoding.type_ids)

['▁Let', "'", 's', '▁test', '▁this', '▁to', 'ken', 'izer', '.', '.', '.', '<sep>', '▁', 'on', '▁', 'a', '▁pair', '▁of', '▁sentence', 's', '!', '<sep>', '<cls>']
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2]


In [62]:
tokenizer.decoder = decoders.Metaspace()

In [63]:
from transformers import PreTrainedTokenizerFast

wrapped_tokenizer = PreTrainedTokenizerFast(
    tokenizer_object=tokenizer,
    bos_token="<s>",
    eos_token="</s>",
    unk_token="<unk>",
    pad_token="<pad>",
    cls_token="<cls>",
    sep_token="<sep>",
    mask_token="<mask>",
    padding_side="left",
)

In [64]:
from transformers import XLNetTokenizerFast

wrapped_tokenizer = XLNetTokenizerFast(tokenizer_object=tokenizer)